# Predictive Modeling Assignment 3
## ISLP Ch-3

#### imports and setup

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as st

### QUESTION # 2
Carefully explain the differences between the KNN classifier and KNN regression methods.

### QUESTION # 9 
This question involves the use of multiple linear regression on the Auto data set. 
#### (a) 
Produce a scatterplot matrix which includes all of the variables in the data set.  

#### (b) 
Compute the matrix of correlations between the variables using the DataFrame.corr() method. .corr() 

#### (c) 
Use the sm.OLS() function to perform a multiple linear regression with mpg as the response and all other variables except name as the predictors. Use the summarize() function to print the results. Comment on the output. For instance: i. Is there a relationship between the predictors and the response? Use the anova_lm() function from statsmodels to answer this question.130 3. Linear Regression ii. Which predictors appear to have a statistically significant relationship to the response? iii. What does the coefficient for the year variable suggest? 

#### (d) 
Produce some of diagnostic plots of the linear regression fit as described in the lab. Comment on any problems you see with the fit. Do the residual plots suggest any unusually large outliers? Does the leverage plot identify any observations with unusually high leverage?

#### (e) 
Fit some models with interactions as described in the lab. Do any interactions appear to be statistically significant? 

#### (f) 
Try a few different transformations of the variables, such as log(X), √X, X2. Comment on your findings.

### QUESTION # 10 

This question should be answered using the Carseats data set. (a) Fit a multiple regression model to predict Sales using Price Urban, and US.

#### (b) 
Provide an interpretation of each coefficient in the model. Be careful—some of the variables in the model are qualitative!


#### (c) 
Write out the model in equation form, being careful to handle the qualitative variables properly.


#### (d) 
For which of the predictors can you reject the null hypothesis H0 : βj = 0?


#### (e) 
On the basis of your response to the previous question, fit a smaller model that only uses the predictors for which there is evidence of association with the outcome.


#### (f) 
How well do the models in (a) and (e) fit the data?


#### (g) 
Using the model from (e), obtain 95 % confidence intervals for the coefficient(s).


#### (h) 
Is there evidence of outliers or high leverage observations in the model from (e)?

### QUESTION # 12

This problem involves simple linear regression without an intercept. 
#### (a) 
Recall that the coefficient estimate βˆ for the linear regression of Y onto X without an intercept is given by (3.38). Under what circumstance is the coefficient estimate for the regression of X onto Y the same as the coefficient estimate for the regression of Y onto X? 

#### (b) 
Generate an example in Python with n = 100 observations in which the coefficient estimate for the regression of X onto Y is different from the coefficient estimate for the regression of Y onto X. 

#### (c) 
Generate an example in Python with n = 100 observations in which the coefficient estimate for the regression of X onto Y is the same as the coefficient estimate for the regression of Y onto X.